In [37]:
import pandas as pd
import glob
import numpy
import csv
from nltk.tokenize import word_tokenize
import xlrd
import re
from nltk import bigrams
from nltk import ngrams
import operator
from collections import Counter
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt

from random import randint
from numpy import array
from numpy import argmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [38]:
path = r'E:\Level 4 Project\dataset\final\preprocess' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
df = pd.concat(li, axis=0, ignore_index=True, sort=False)

In [39]:
df.head()

,Date,Tweet
0,2011-01-01 23:58:38+00:00,Music for space tourism Download rapidshare m...
1,2011-01-01 23:58:26+00:00,Rajasthan as Tourism Destinations http://bit....
2,2011-01-01 23:57:34+00:00,"California decriminalizes pot, under a certai..."
3,2011-01-01 23:57:28+00:00,NPL Tourism year begins with fair in Lake Cit...
4,2011-01-01 23:54:57+00:00,Though I suppose a better comparison would be...


In [40]:
df.shape

(10295057, 2)

In [41]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

def preprocess_fornothash(s, lowercase=True):
    tokens = tokenize_fornothash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_fornothash(s):
    return tokens_re_fornothash.findall(s)


regex_str_fornothash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    #r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_fornothash = re.compile(r'(' + '|'.join(regex_str_fornothash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

def preprocess_forhash(s, lowercase=True):
    tokens = tokenize_forhash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_forhash(s):
    return tokens_re_forhash.findall(s)

regex_str_forhash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_forhash = re.compile(r'(' + '|'.join(regex_str_forhash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

punctuation = list(string.punctuation)
unnecssary = ['The','tourism','I','rt', 'via','...','â','click']
unnecssary_hash = ['#','#tourism']
ascii_words = ['\x92','\x96']
years = ['2018','2017','2016','2015','2014','2013']
numbers = ['nine']
stop = stopwords.words('english') + punctuation + unnecssary + ascii_words + years + unnecssary_hash + numbers

In [42]:
import datetime

In [43]:
s = datetime.date(2011, 1, 1).strftime('%Y-%m-%d')
print(s)

2011-01-01


In [52]:
today = datetime.datetime.strptime("2011-01-01", '%Y-%m-%d').date()

count_all = []
counter = [] 
daysCount =0
i = 0
text = ''
count_all.append(Counter())
excel_array = []

for index, row in df.iterrows():
    
    #nextDay =  pd.to_datetime("2011-01-01") + pd.DateOffset(days=daysCount+1)
    #afterNextDay =  pd.to_datetime("2011-01-01") + pd.DateOffset(days=daysCount+2)
    if(today != pd.to_datetime(row['Date'][:10]).date()):
        counter.append(i)
        if(i != 0):
            print('date - ',today.strftime('%Y-%m-%d'), ' i - ',i)
            #print([today.strftime('%Y-%m-%d'),count_all[daysCount].most_common(10)])
            excel_array.append([today.strftime('%Y-%m-%d'),count_all[daysCount].most_common(10)])
        daysCount += 1
        i = 0
        text = ''
        count_all.append(Counter())

    i = i+1
    text = row.Tweet
    terms_all = [term for term in preprocess_forhash(text)
        if term.startswith('#') if term not in stop]
    count_all[daysCount].update(terms_all)
    today =  pd.to_datetime(row['Date'][:10]).date()

date -  2011-01-01  i -  973
date -  2011-01-02  i -  1424
date -  2011-01-03  i -  2427
date -  2011-01-04  i -  2578
date -  2011-01-05  i -  2742
date -  2011-01-06  i -  2487
date -  2011-01-07  i -  2410
date -  2011-01-08  i -  1294
date -  2011-01-09  i -  1532
date -  2011-01-10  i -  2467
date -  2011-01-11  i -  2562
date -  2011-01-12  i -  2484
date -  2011-01-13  i -  2471
date -  2011-01-14  i -  2658
date -  2011-01-15  i -  1658
date -  2011-01-16  i -  2037
date -  2011-01-17  i -  3092
date -  2011-01-18  i -  2897
date -  2011-01-19  i -  2852
date -  2011-01-20  i -  2614
date -  2011-01-21  i -  2918
date -  2011-01-22  i -  1734
date -  2011-01-23  i -  1643
date -  2011-01-24  i -  2721
date -  2011-01-25  i -  2587
date -  2011-01-26  i -  2629
date -  2011-01-27  i -  2584
date -  2011-01-28  i -  2820
date -  2011-01-29  i -  1715
date -  2011-01-30  i -  1677
date -  2011-02-01  i -  3523
date -  2011-02-02  i -  3160
date -  2011-02-03  i -  2860
date -  201

date -  2011-10-12  i -  3561
date -  2011-10-13  i -  3622
date -  2011-10-14  i -  3599
date -  2011-10-15  i -  2815
date -  2011-10-16  i -  2533
date -  2011-10-17  i -  3589
date -  2011-10-18  i -  3582
date -  2011-10-19  i -  3560
date -  2011-10-20  i -  3579
date -  2011-10-21  i -  3467
date -  2011-10-22  i -  2458
date -  2011-10-23  i -  2671
date -  2011-10-24  i -  3604
date -  2011-10-25  i -  3551
date -  2011-10-26  i -  3546
date -  2011-10-27  i -  3621
date -  2011-10-28  i -  3591
date -  2011-10-29  i -  2882
date -  2011-10-30  i -  3071
date -  2011-11-01  i -  3590
date -  2011-11-02  i -  3621
date -  2011-11-03  i -  3526
date -  2011-11-04  i -  3387
date -  2011-11-05  i -  2959
date -  2011-11-06  i -  3051
date -  2011-11-07  i -  3689
date -  2011-11-08  i -  3612
date -  2011-11-09  i -  3595
date -  2011-11-10  i -  3599
date -  2011-11-11  i -  3603
date -  2011-11-12  i -  2944
date -  2011-11-13  i -  2674
date -  2011-11-14  i -  3584
date -  20

date -  2012-07-21  i -  2523
date -  2012-07-22  i -  2779
date -  2012-07-23  i -  3397
date -  2012-07-24  i -  3475
date -  2012-07-25  i -  3527
date -  2012-07-26  i -  3549
date -  2012-07-27  i -  3540
date -  2012-07-28  i -  2480
date -  2012-07-29  i -  2318
date -  2012-07-30  i -  3398
date -  2012-08-01  i -  3424
date -  2012-08-02  i -  3352
date -  2012-08-03  i -  3400
date -  2012-08-04  i -  2502
date -  2012-08-05  i -  2196
date -  2012-08-06  i -  3418
date -  2012-08-07  i -  3495
date -  2012-08-08  i -  3456
date -  2012-08-09  i -  3277
date -  2012-08-10  i -  3395
date -  2012-08-11  i -  2688
date -  2012-08-12  i -  2486
date -  2012-08-13  i -  3414
date -  2012-08-14  i -  3476
date -  2012-08-15  i -  3509
date -  2012-08-16  i -  3470
date -  2012-08-17  i -  3367
date -  2012-08-18  i -  2678
date -  2012-08-19  i -  2245
date -  2012-08-20  i -  3430
date -  2012-08-21  i -  3386
date -  2012-08-22  i -  3535
date -  2012-08-23  i -  3463
date -  20

date -  2013-05-01  i -  3449
date -  2013-05-02  i -  3210
date -  2013-05-03  i -  3505
date -  2013-05-04  i -  3410
date -  2013-05-05  i -  3426
date -  2013-05-06  i -  3551
date -  2013-05-07  i -  3539
date -  2013-05-08  i -  3599
date -  2013-05-09  i -  3540
date -  2013-05-10  i -  3450
date -  2013-05-11  i -  3367
date -  2013-05-12  i -  3031
date -  2013-05-13  i -  3289
date -  2013-05-14  i -  3436
date -  2013-05-15  i -  3449
date -  2013-05-16  i -  3469
date -  2013-05-17  i -  3479
date -  2013-05-18  i -  3374
date -  2013-05-19  i -  2767
date -  2013-05-20  i -  3474
date -  2013-05-21  i -  3460
date -  2013-05-22  i -  3462
date -  2013-05-23  i -  3421
date -  2013-05-24  i -  3386
date -  2013-05-25  i -  2829
date -  2013-05-26  i -  2838
date -  2013-05-27  i -  3309
date -  2013-05-28  i -  3442
date -  2013-05-29  i -  3407
date -  2013-05-30  i -  3118
date -  2013-06-01  i -  2966
date -  2013-06-02  i -  2925
date -  2013-06-03  i -  3436
date -  20

date -  2014-02-08  i -  3223
date -  2014-02-09  i -  3279
date -  2014-02-10  i -  3403
date -  2014-02-11  i -  3301
date -  2014-02-12  i -  3404
date -  2014-02-13  i -  3406
date -  2014-02-14  i -  3357
date -  2014-02-15  i -  2989
date -  2014-02-16  i -  3013
date -  2014-02-17  i -  3230
date -  2014-02-18  i -  3345
date -  2014-02-19  i -  3341
date -  2014-02-20  i -  3222
date -  2014-02-21  i -  3462
date -  2014-02-22  i -  3228
date -  2014-02-23  i -  2923
date -  2014-02-24  i -  3353
date -  2014-02-25  i -  3306
date -  2014-02-26  i -  3304
date -  2014-02-27  i -  3313
date -  2014-03-01  i -  3149
date -  2014-03-02  i -  2903
date -  2014-03-03  i -  3461
date -  2014-03-04  i -  3367
date -  2014-03-05  i -  3491
date -  2014-03-06  i -  3346
date -  2014-03-07  i -  3376
date -  2014-03-08  i -  3008
date -  2014-03-09  i -  3168
date -  2014-03-10  i -  3383
date -  2014-03-11  i -  3360
date -  2014-03-12  i -  3383
date -  2014-03-13  i -  3349
date -  20

...

date -  2016-06-08  i -  3598
date -  2016-06-09  i -  3484
date -  2016-06-10  i -  3497
date -  2016-06-11  i -  3471
date -  2016-06-12  i -  3374
date -  2016-06-13  i -  3526
date -  2016-06-14  i -  3559
date -  2016-06-15  i -  3436
date -  2016-06-16  i -  3498
date -  2016-06-17  i -  3392
date -  2016-06-18  i -  3497
date -  2016-06-19  i -  3277
date -  2016-06-20  i -  3276
date -  2016-06-21  i -  3423
date -  2016-06-22  i -  3414
date -  2016-06-23  i -  3484
date -  2016-06-24  i -  3528
date -  2016-06-25  i -  3345
date -  2016-06-26  i -  3277
date -  2016-06-27  i -  3485
date -  2016-06-28  i -  3533
date -  2016-06-29  i -  3498
date -  2016-07-01  i -  1876
date -  2016-07-02  i -  3323
date -  2016-07-03  i -  2677
date -  2016-07-04  i -  3138
date -  2016-07-05  i -  3440
date -  2016-07-06  i -  3425
date -  2016-07-07  i -  3482
date -  2016-07-08  i -  3390
date -  2016-07-09  i -  3269
date -  2016-07-10  i -  3265
date -  2016-07-11  i -  3406
date -  20

date -  2017-03-19  i -  3234
date -  2017-03-20  i -  3350
date -  2017-03-21  i -  3451
date -  2017-03-22  i -  3473
date -  2017-03-23  i -  3449
date -  2017-03-24  i -  3462
date -  2017-03-25  i -  2696
date -  2017-03-26  i -  3001
date -  2017-03-27  i -  3489
date -  2017-03-28  i -  3506
date -  2017-03-29  i -  3546
date -  2017-03-30  i -  3531
date -  2017-04-01  i -  2055
date -  2017-04-02  i -  3542
date -  2017-04-03  i -  3549
date -  2017-04-04  i -  3568
date -  2017-04-05  i -  3454
date -  2017-04-06  i -  3438
date -  2017-04-07  i -  3508
date -  2017-04-08  i -  3369
date -  2017-04-09  i -  3174
date -  2017-04-10  i -  3414
date -  2017-04-11  i -  3294
date -  2017-04-12  i -  3414
date -  2017-04-13  i -  3451
date -  2017-04-14  i -  3355
date -  2017-04-15  i -  3335
date -  2017-04-16  i -  3185
date -  2017-04-17  i -  3463
date -  2017-04-18  i -  3487
date -  2017-04-19  i -  3535
date -  2017-04-20  i -  3471
date -  2017-04-21  i -  3451
date -  20

date -  2017-12-28  i -  3416
date -  2017-12-29  i -  3450
date -  2017-12-30  i -  3247
date -  2018-01-01  i -  2691
date -  2018-01-02  i -  3435
date -  2018-01-03  i -  3418
date -  2018-01-04  i -  3193
date -  2018-01-05  i -  3461
date -  2018-01-06  i -  3146
date -  2018-01-07  i -  3085
date -  2018-01-08  i -  3384
date -  2018-01-09  i -  3404
date -  2018-01-10  i -  3556
date -  2018-01-11  i -  3512
date -  2018-01-12  i -  3364
date -  2018-01-13  i -  3370
date -  2018-01-14  i -  1970
date -  2018-01-15  i -  3443
date -  2018-01-16  i -  3500
date -  2018-01-17  i -  3441
date -  2018-01-18  i -  3507
date -  2018-01-19  i -  3385
date -  2018-01-20  i -  3204
date -  2018-01-21  i -  1600
date -  2018-01-22  i -  3377
date -  2018-01-23  i -  3436
date -  2018-01-24  i -  3643
date -  2018-01-25  i -  3575
date -  2018-01-26  i -  3311
date -  2018-01-27  i -  2925
date -  2018-01-28  i -  1112
date -  2018-01-29  i -  3442
date -  2018-01-30  i -  3482
date -  20

date -  2018-10-08  i -  3460
date -  2018-10-09  i -  3489
date -  2018-10-10  i -  3518
date -  2018-10-11  i -  3520
date -  2018-10-12  i -  3472
date -  2018-10-13  i -  3346
date -  2018-10-14  i -  3384
date -  2018-10-15  i -  3534
date -  2018-10-16  i -  3413
date -  2018-10-17  i -  3500
date -  2018-10-18  i -  3442
date -  2018-10-19  i -  3199
date -  2018-10-20  i -  3282
date -  2018-10-21  i -  3425
date -  2018-10-22  i -  3424
date -  2018-10-23  i -  3530
date -  2018-10-24  i -  3429
date -  2018-10-25  i -  3386
date -  2018-10-26  i -  3407
date -  2018-10-27  i -  3322
date -  2018-10-28  i -  3284
date -  2018-10-29  i -  3499
date -  2018-10-30  i -  3589
date -  2018-11-01  i -  3528
date -  2018-11-02  i -  3542
date -  2018-11-03  i -  3311
date -  2018-11-04  i -  3362
date -  2018-11-05  i -  3442
date -  2018-11-06  i -  3461
date -  2018-11-07  i -  3496
date -  2018-11-08  i -  3432
date -  2018-11-09  i -  3432
date -  2018-11-10  i -  3321
date -  20

date -  2019-07-18  i -  3490
date -  2019-07-19  i -  3416
date -  2019-07-20  i -  3254
date -  2019-07-21  i -  3276
date -  2019-07-22  i -  3283
date -  2019-07-23  i -  3286
date -  2019-07-24  i -  3460
date -  2019-07-25  i -  3311
date -  2019-07-26  i -  3422
date -  2019-07-27  i -  3321
date -  2019-07-28  i -  3217
date -  2019-07-29  i -  3333
date -  2019-07-30  i -  3379
date -  2019-08-01  i -  3429
date -  2019-08-02  i -  3388
date -  2019-08-03  i -  3312
date -  2019-08-04  i -  3332
date -  2019-08-05  i -  3379
date -  2019-08-06  i -  3516
date -  2019-08-07  i -  3510
date -  2019-08-08  i -  3467
date -  2019-08-09  i -  3186
date -  2019-08-10  i -  3346
date -  2019-08-11  i -  3269
date -  2019-08-12  i -  3469
date -  2019-08-13  i -  3335
date -  2019-08-14  i -  3470
date -  2019-08-15  i -  3461
date -  2019-08-16  i -  3460
date -  2019-08-17  i -  3222
date -  2019-08-18  i -  3286
date -  2019-08-19  i -  3327
date -  2019-08-20  i -  3393
date -  20

In [44]:
today = datetime.datetime.strptime("2011-01-01", '%Y-%m-%d').date()
counter = 0

for index, row in df.iterrows():
    
    #nextDay =  pd.to_datetime("2011-01-01") + pd.DateOffset(days=daysCount+1)
    #afterNextDay =  pd.to_datetime("2011-01-01") + pd.DateOffset(days=daysCount+2)
#     if()
#     print(today.strftime('%m/%d/%Y'))
    if(today != pd.to_datetime(row['Date'][:10]).date()):
        days_difference = (pd.to_datetime(row['Date'][:10]).date() - today).days
        print(days_difference,'today - ' ,today, ' counter -'  ,counter)
        counter = 0
        if(days_difference != 1):
            print("Hello")
    today =  pd.to_datetime(row['Date'][:10]).date()
    counter = counter+1

1 today -  2011-01-01  counter - 973
1 today -  2011-01-02  counter - 1424
1 today -  2011-01-03  counter - 2427
1 today -  2011-01-04  counter - 2578
1 today -  2011-01-05  counter - 2742
1 today -  2011-01-06  counter - 2487
1 today -  2011-01-07  counter - 2410
1 today -  2011-01-08  counter - 1294
1 today -  2011-01-09  counter - 1532
1 today -  2011-01-10  counter - 2467
1 today -  2011-01-11  counter - 2562
1 today -  2011-01-12  counter - 2484
1 today -  2011-01-13  counter - 2471
1 today -  2011-01-14  counter - 2658
1 today -  2011-01-15  counter - 1658
1 today -  2011-01-16  counter - 2037
1 today -  2011-01-17  counter - 3092
1 today -  2011-01-18  counter - 2897
1 today -  2011-01-19  counter - 2852
1 today -  2011-01-20  counter - 2614
1 today -  2011-01-21  counter - 2918
1 today -  2011-01-22  counter - 1734
1 today -  2011-01-23  counter - 1643
1 today -  2011-01-24  counter - 2721
1 today -  2011-01-25  counter - 2587
1 today -  2011-01-26  counter - 2629
1 today -  20

1 today -  2011-08-11  counter - 3605
1 today -  2011-08-12  counter - 3565
1 today -  2011-08-13  counter - 2239
1 today -  2011-08-14  counter - 2063
1 today -  2011-08-15  counter - 3496
1 today -  2011-08-16  counter - 3275
1 today -  2011-08-17  counter - 3668
1 today -  2011-08-18  counter - 3525
1 today -  2011-08-19  counter - 3415
1 today -  2011-08-20  counter - 2678
1 today -  2011-08-21  counter - 2325
1 today -  2011-08-22  counter - 3563
1 today -  2011-08-23  counter - 3663
1 today -  2011-08-24  counter - 3649
1 today -  2011-08-25  counter - 3422
1 today -  2011-08-26  counter - 3458
1 today -  2011-08-27  counter - 2155
1 today -  2011-08-28  counter - 2042
1 today -  2011-08-29  counter - 3177
2 today -  2011-08-30  counter - 3579
Hello
1 today -  2011-09-01  counter - 3656
1 today -  2011-09-02  counter - 3587
1 today -  2011-09-03  counter - 2978
1 today -  2011-09-04  counter - 2947
1 today -  2011-09-05  counter - 3622
1 today -  2011-09-06  counter - 3526
1 toda

1 today -  2012-03-21  counter - 3577
1 today -  2012-03-22  counter - 3510
1 today -  2012-03-23  counter - 3370
1 today -  2012-03-24  counter - 3131
1 today -  2012-03-25  counter - 3042
1 today -  2012-03-26  counter - 3531
1 today -  2012-03-27  counter - 3546
1 today -  2012-03-28  counter - 3537
1 today -  2012-03-29  counter - 3524
2 today -  2012-03-30  counter - 3447
Hello
1 today -  2012-04-01  counter - 2425
1 today -  2012-04-02  counter - 3497
1 today -  2012-04-03  counter - 3461
1 today -  2012-04-04  counter - 3491
1 today -  2012-04-05  counter - 3476
1 today -  2012-04-06  counter - 3408
1 today -  2012-04-07  counter - 2965
1 today -  2012-04-08  counter - 2339
1 today -  2012-04-09  counter - 3370
1 today -  2012-04-10  counter - 3475
1 today -  2012-04-11  counter - 3454
1 today -  2012-04-12  counter - 3479
1 today -  2012-04-13  counter - 3446
1 today -  2012-04-14  counter - 3076
1 today -  2012-04-15  counter - 2980
1 today -  2012-04-16  counter - 3498
1 toda

1 today -  2012-10-29  counter - 2612
2 today -  2012-10-30  counter - 2594
Hello
1 today -  2012-11-01  counter - 2639
1 today -  2012-11-02  counter - 3459
1 today -  2012-11-03  counter - 2653
1 today -  2012-11-04  counter - 2730
1 today -  2012-11-05  counter - 3447
1 today -  2012-11-06  counter - 3417
1 today -  2012-11-07  counter - 3597
1 today -  2012-11-08  counter - 3504
1 today -  2012-11-09  counter - 3516
1 today -  2012-11-10  counter - 3005
1 today -  2012-11-11  counter - 2517
1 today -  2012-11-12  counter - 3423
1 today -  2012-11-13  counter - 3431
1 today -  2012-11-14  counter - 2947
1 today -  2012-11-15  counter - 3498
1 today -  2012-11-16  counter - 3464
1 today -  2012-11-17  counter - 2513
1 today -  2012-11-18  counter - 2718
1 today -  2012-11-19  counter - 3505
1 today -  2012-11-20  counter - 3406
1 today -  2012-11-21  counter - 3401
1 today -  2012-11-22  counter - 3431
1 today -  2012-11-23  counter - 3402
1 today -  2012-11-24  counter - 2845
1 toda

1 today -  2013-06-09  counter - 3071
1 today -  2013-06-10  counter - 3397
1 today -  2013-06-11  counter - 3534
1 today -  2013-06-12  counter - 3402
1 today -  2013-06-13  counter - 3341
1 today -  2013-06-14  counter - 3295
1 today -  2013-06-15  counter - 3037
1 today -  2013-06-16  counter - 2249
1 today -  2013-06-17  counter - 3334
1 today -  2013-06-18  counter - 3440
1 today -  2013-06-19  counter - 3232
1 today -  2013-06-20  counter - 3304
1 today -  2013-06-21  counter - 3166
1 today -  2013-06-22  counter - 2857
1 today -  2013-06-23  counter - 3345
1 today -  2013-06-24  counter - 3360
1 today -  2013-06-25  counter - 3345
1 today -  2013-06-26  counter - 3380
1 today -  2013-06-27  counter - 3285
1 today -  2013-06-28  counter - 3275
2 today -  2013-06-29  counter - 2982
Hello
1 today -  2013-07-01  counter - 3433
1 today -  2013-07-02  counter - 3372
1 today -  2013-07-03  counter - 3473
1 today -  2013-07-04  counter - 3401
1 today -  2013-07-05  counter - 3274
1 toda

...

1 today -  2017-02-04  counter - 3427
1 today -  2017-02-05  counter - 3074
1 today -  2017-02-06  counter - 3451
1 today -  2017-02-07  counter - 3474
1 today -  2017-02-08  counter - 3571
1 today -  2017-02-09  counter - 3443
1 today -  2017-02-10  counter - 3455
1 today -  2017-02-11  counter - 3369
1 today -  2017-02-12  counter - 3095
1 today -  2017-02-13  counter - 3455
1 today -  2017-02-14  counter - 3460
1 today -  2017-02-15  counter - 3544
1 today -  2017-02-16  counter - 3483
1 today -  2017-02-17  counter - 3462
1 today -  2017-02-18  counter - 3274
1 today -  2017-02-19  counter - 3303
1 today -  2017-02-20  counter - 3413
1 today -  2017-02-21  counter - 3505
1 today -  2017-02-22  counter - 3559
1 today -  2017-02-23  counter - 3292
1 today -  2017-02-24  counter - 3516
1 today -  2017-02-25  counter - 3423
1 today -  2017-02-26  counter - 3431
2 today -  2017-02-27  counter - 3494
Hello
1 today -  2017-03-01  counter - 3469
1 today -  2017-03-02  counter - 3504
1 toda

1 today -  2017-09-14  counter - 3439
1 today -  2017-09-15  counter - 3470
1 today -  2017-09-16  counter - 3296
1 today -  2017-09-17  counter - 3316
1 today -  2017-09-18  counter - 3400
1 today -  2017-09-19  counter - 3476
1 today -  2017-09-20  counter - 3485
1 today -  2017-09-21  counter - 3466
1 today -  2017-09-22  counter - 3389
1 today -  2017-09-23  counter - 3354
1 today -  2017-09-24  counter - 2538
1 today -  2017-09-25  counter - 3407
1 today -  2017-09-26  counter - 3532
1 today -  2017-09-27  counter - 3471
1 today -  2017-09-28  counter - 3467
2 today -  2017-09-29  counter - 3409
Hello
1 today -  2017-10-01  counter - 3377
1 today -  2017-10-02  counter - 3537
1 today -  2017-10-03  counter - 3472
1 today -  2017-10-04  counter - 3416
1 today -  2017-10-05  counter - 3498
1 today -  2017-10-06  counter - 3499
1 today -  2017-10-07  counter - 3317
1 today -  2017-10-08  counter - 3262
1 today -  2017-10-09  counter - 3275
1 today -  2017-10-10  counter - 3444
1 toda

1 today -  2018-04-25  counter - 3353
1 today -  2018-04-26  counter - 3484
1 today -  2018-04-27  counter - 3293
1 today -  2018-04-28  counter - 3365
2 today -  2018-04-29  counter - 3446
Hello
1 today -  2018-05-01  counter - 3499
1 today -  2018-05-02  counter - 3400
1 today -  2018-05-03  counter - 3367
1 today -  2018-05-04  counter - 3497
1 today -  2018-05-05  counter - 3274
1 today -  2018-05-06  counter - 3291
1 today -  2018-05-07  counter - 3669
1 today -  2018-05-08  counter - 3582
1 today -  2018-05-09  counter - 3565
1 today -  2018-05-10  counter - 3537
1 today -  2018-05-11  counter - 3547
1 today -  2018-05-12  counter - 3418
1 today -  2018-05-13  counter - 3263
1 today -  2018-05-14  counter - 3503
1 today -  2018-05-15  counter - 3463
1 today -  2018-05-16  counter - 3442
1 today -  2018-05-17  counter - 3481
1 today -  2018-05-18  counter - 3439
1 today -  2018-05-19  counter - 3554
1 today -  2018-05-20  counter - 3423
1 today -  2018-05-21  counter - 3418
1 toda

1 today -  2018-12-04  counter - 3428
1 today -  2018-12-05  counter - 3233
1 today -  2018-12-06  counter - 3166
1 today -  2018-12-07  counter - 3201
1 today -  2018-12-08  counter - 3332
1 today -  2018-12-09  counter - 3387
1 today -  2018-12-10  counter - 3434
1 today -  2018-12-11  counter - 3347
1 today -  2018-12-12  counter - 3375
1 today -  2018-12-13  counter - 3413
1 today -  2018-12-14  counter - 2970
1 today -  2018-12-15  counter - 3285
1 today -  2018-12-16  counter - 3042
1 today -  2018-12-17  counter - 3440
1 today -  2018-12-18  counter - 3479
1 today -  2018-12-19  counter - 3329
1 today -  2018-12-20  counter - 3120
1 today -  2018-12-21  counter - 3416
1 today -  2018-12-22  counter - 3235
1 today -  2018-12-23  counter - 3035
1 today -  2018-12-24  counter - 3302
1 today -  2018-12-25  counter - 2612
1 today -  2018-12-26  counter - 3161
1 today -  2018-12-27  counter - 3427
1 today -  2018-12-28  counter - 3453
1 today -  2018-12-29  counter - 3305
2 today -  2

1 today -  2019-07-14  counter - 3271
1 today -  2019-07-15  counter - 3395
1 today -  2019-07-16  counter - 3439
1 today -  2019-07-17  counter - 3417
1 today -  2019-07-18  counter - 3490
1 today -  2019-07-19  counter - 3416
1 today -  2019-07-20  counter - 3254
1 today -  2019-07-21  counter - 3276
1 today -  2019-07-22  counter - 3283
1 today -  2019-07-23  counter - 3286
1 today -  2019-07-24  counter - 3460
1 today -  2019-07-25  counter - 3311
1 today -  2019-07-26  counter - 3422
1 today -  2019-07-27  counter - 3321
1 today -  2019-07-28  counter - 3217
1 today -  2019-07-29  counter - 3333
2 today -  2019-07-30  counter - 3379
Hello
1 today -  2019-08-01  counter - 3429
1 today -  2019-08-02  counter - 3388
1 today -  2019-08-03  counter - 3312
1 today -  2019-08-04  counter - 3332
1 today -  2019-08-05  counter - 3379
1 today -  2019-08-06  counter - 3516
1 today -  2019-08-07  counter - 3510
1 today -  2019-08-08  counter - 3467
1 today -  2019-08-09  counter - 3186
1 toda

In [55]:
for x in excel_array:
    print(x[0])
    

2011-01-01
2011-01-02
2011-01-03
2011-01-04
2011-01-05
2011-01-06
2011-01-07
2011-01-08
2011-01-09
2011-01-10
2011-01-11
2011-01-12
2011-01-13
2011-01-14
2011-01-15
2011-01-16
2011-01-17
2011-01-18
2011-01-19
2011-01-20
2011-01-21
2011-01-22
2011-01-23
2011-01-24
2011-01-25
2011-01-26
2011-01-27
2011-01-28
2011-01-29
2011-01-30
2011-02-01
2011-02-02
2011-02-03
2011-02-04
2011-02-05
2011-02-06
2011-02-07
2011-02-08
2011-02-09
2011-02-10
2011-02-11
2011-02-12
2011-02-13
2011-02-14
2011-02-15
2011-02-16
2011-02-17
2011-02-18
2011-02-19
2011-02-20
2011-02-21
2011-02-22
2011-02-23
2011-02-24
2011-02-25
2011-02-26
2011-02-27
2011-03-01
2011-03-02
2011-03-03
2011-03-04
2011-03-05
2011-03-06
2011-03-07
2011-03-08
2011-03-09
2011-03-10
2011-03-11
2011-03-12
2011-03-13
2011-03-14
2011-03-15
2011-03-16
2011-03-17
2011-03-18
2011-03-19
2011-03-20
2011-03-21
2011-03-22
2011-03-23
2011-03-24
2011-03-25
2011-03-26
2011-03-27
2011-03-28
2011-03-29
2011-03-30
2011-04-01
2011-04-02
2011-04-03
2011-04-04

2015-09-29
2015-10-01
2015-10-02
2015-10-03
2015-10-04
2015-10-05
2015-10-06
2015-10-07
2015-10-08
2015-10-09
2015-10-10
2015-10-11
2015-10-12
2015-10-13
2015-10-14
2015-10-15
2015-10-16
2015-10-17
2015-10-18
2015-10-19
2015-10-20
2015-10-21
2015-10-22
2015-10-23
2015-10-24
2015-10-25
2015-10-26
2015-10-27
2015-10-28
2015-10-29
2015-10-30
2015-11-01
2015-11-02
2015-11-03
2015-11-04
2015-11-05
2015-11-06
2015-11-07
2015-11-08
2015-11-09
2015-11-10
2015-11-11
2015-11-12
2015-11-13
2015-11-14
2015-11-15
2015-11-16
2015-11-17
2015-11-18
2015-11-19
2015-11-20
2015-11-21
2015-11-22
2015-11-23
2015-11-24
2015-11-25
2015-11-26
2015-11-27
2015-11-28
2015-11-29
2015-12-01
2015-12-02
2015-12-03
2015-12-04
2015-12-05
2015-12-06
2015-12-07
2015-12-08
2015-12-09
2015-12-10
2015-12-11
2015-12-12
2015-12-13
2015-12-14
2015-12-15
2015-12-16
2015-12-17
2015-12-18
2015-12-19
2015-12-20
2015-12-21
2015-12-22
2015-12-23
2015-12-24
2015-12-25
2015-12-26
2015-12-27
2015-12-28
2015-12-29
2015-12-30
2016-01-01

2017-12-13
2017-12-14
2017-12-15
2017-12-16
2017-12-17
2017-12-18
2017-12-19
2017-12-20
2017-12-21
2017-12-22
2017-12-23
2017-12-24
2017-12-25
2017-12-26
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-02-01
2018-02-02
2018-02-03
2018-02-04
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26
2018-02-27
2018-03-01
2018-03-02
2018-03-03
2018-03-04
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16

In [56]:
with open('dataset/process/csv-test/all.csv', 'w',encoding="utf-8",newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in excel_array:
        writer.writerow([row[0], row[1]])   